In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

## Load cleaned dataset 

In [2]:
df_prices = pd.read_csv('./EDA/data/price_clean.csv')
df_calendar = pd.read_csv('./EDA/data/calendar_clean.csv')
df_sales = pd.read_csv('./EDA/data/sales_clean.csv')

### Check size and headers of datasets

In [3]:
print(df_prices.shape)
df_prices.drop(columns=["category","yearweek"], axis=1, inplace=True)
df_prices.head(1)

(45986376, 6)


,date,item,store_code,sell_price
0,2013-07-16,ACCESORIES_1_001,NYC_1,12.7414


In [4]:
#Delete unnecesary columns
print(df_calendar.shape)
df_calendar.drop(columns=["Unnamed: 0","weekday",'d','weekday_int'], axis=1, inplace=True)
df_calendar.head(1)

(1913, 6)


,date,event
0,2011-01-29,no event


In [5]:
print(df_sales.shape)
df_sales.head(1)

(58327370, 4)


,date,id,store,sales
0,2011-01-29,ACCESORIES_1_001_NYC_1,Greenwich_Village,0


### I do merge df_prices with df_calendar

In [6]:
df=pd.merge(df_prices, df_calendar, on=['date'],how='left')
df.head(1)

,date,item,store_code,sell_price,event
0,2013-07-16,ACCESORIES_1_001,NYC_1,12.7414,no event


### Making new colunm so to merge by it

In [7]:
df['id']=df['item']+"_"+df['store_code']
df.tail(1)

,date,item,store_code,sell_price,event,id
45986375,2016-04-24,SUPERMARKET_3_827,PHI_3,1.2,no event,SUPERMARKET_3_827_PHI_3


### Delete unnecesary columns for the **df** merge

In [8]:
df.drop(columns=['item','store_code'], axis=1, inplace=True)
df.head(1)

,date,sell_price,event,id
0,2013-07-16,12.7414,no event,ACCESORIES_1_001_NYC_1


In [9]:
df= df[['date','id','sell_price','event']] #i ordered my columns

### I perform the final merge of  **df** with **df_sales**, obtaining te main dataset

In [10]:
df_market=pd.merge(df_sales, df, on=['date','id'],how='left')
df_market.head(1)

,date,id,store,sales,sell_price,event
0,2011-01-29,ACCESORIES_1_001_NYC_1,Greenwich_Village,0,NaN,NaN


### Check nulls

In [11]:
df_market.isnull().sum()

date                 0
id                   0
store                0
sales                0
sell_price    12340994
event         12340994
dtype: int64

### This nulls  on sell_price column as we consider if there isn´t price it is because the item hadn´t arrive yet to the store

In [12]:
df_market.dropna(subset=(["sell_price"]), inplace = True) # delete nulls 

In [13]:
print(df_market.shape)
df_market.head(2)

(45986376, 6)


,date,id,store,sales,sell_price,event
7,2011-01-29,ACCESORIES_1_008_NYC_1,Greenwich_Village,12,0.6118,no event
8,2011-01-29,ACCESORIES_1_009_NYC_1,Greenwich_Village,2,2.0748,no event


### Create income column for **df_market**

In [14]:
df_market['income']=df_market['sales']*df_market['sell_price']

In [15]:
df_market.head(1)

,date,id,store,sales,sell_price,event,income
7,2011-01-29,ACCESORIES_1_008_NYC_1,Greenwich_Village,12,0.6118,no event,7.3416


In [16]:
df_market['date'] = pd.to_datetime(df_market['date'], format='%Y-%m-%d')

In [17]:
df_market.set_index('date', inplace=True)
df_market.head(1)

,id,store,sales,sell_price,event,income
date,,,,,,
2011-01-29,ACCESORIES_1_008_NYC_1,Greenwich_Village,12,0.6118,no event,7.3416


In [18]:
file_name = "df_market.csv" #daily
df_market.to_csv(file_name)

### Gruop in mounths for work Visualization

In [19]:
df_market_bi = df_market.groupby(['id']).resample('M').agg(
    store = ('store',lambda x: x.mode()[0] if not x.mode().empty else None),
    unid=('sales', 'sum'),        # Sumar las ventas mensuales
    precio_prom=('sell_price', 'mean'),
    income=('income','sum')
).reset_index()

In [20]:
print(df_market_bi.shape)
df_market_bi.head(2)

(1536776, 6)


,id,date,store,unid,precio_prom,income
0,ACCESORIES_1_001_BOS_1,2013-07-31,South_End,10,11.753875,116.8804
1,ACCESORIES_1_001_BOS_1,2013-08-31,South_End,3,10.985800,32.9574


In [21]:
file_name = "df_market_bi.csv" #mountly
df_market_bi.to_csv(file_name)